In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as mae
import gc
import glob
import random
from tqdm import tqdm

In [25]:
df = pd.read_csv('/content/drive/MyDrive/kaggle_venti/data/train.csv')
unique_pressures = df["pressure"].unique()
df_ = df[df['u_out'] == 0].reset_index(drop=True)
#cs = df['C'].unique()
#rs = df['R'].unique()
u_out = df['u_out'].values
targets = df['pressure'].values[u_out == 0]
rs = df_['R'].values

In [26]:
def make_oof_features(paths):
  oofs = []
  for path in paths:
      if '.csv' in path:
          try:
              oof_ = pd.read_csv(path, usecols=['pred']).reset_index(drop=True).values.reshape(-1)
              
          except:
              oof_ = pd.read_csv(path, usecols=['oof']).reset_index(drop=True).values.reshape(-1)
              print(type(oof_))
      else:
          try:
            oof_ = np.load(path).reshape(-1)
          except:
            oof_ = np.load(path).reshape(-1)
      print(f'loaded {path}')
      oofs.append(oof_)
  oof = np.array(oofs).transpose()
  print(f'The shape is {oof.shape}')
  print('')
  return oof

In [27]:
def make_pred_features(paths):
  oofs = []
  for path in paths:
      if '.csv' in path:
          oof_ = pd.read_csv(path, usecols=['pressure']).values.reshape(-1)
          #print(oof_.shape)
          print(f'loaded {path}')
      else:
          oof_ = np.load(path)
          #print(oof_.shape)
          print(f'loaded {path}')
      oofs.append(oof_)
  oof = np.array(oofs).transpose()
  print(f'The shape is {oof.shape}')
  print('')
  return oof

In [35]:
oof_paths = [#'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_cv0.17652843408959074.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_cv0.17823729046894227.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.17542680160619267.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_060_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_062_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_063_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_064_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_065_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.17969709519240776.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/todasan_oof_exp070.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_090_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_091_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp022.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp027.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp030.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp031.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp032.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_53_2.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/todasan_oof_exp080.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_toda68_classification_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.17454632734884412.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp039.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp040.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp041.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp046.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp047.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp049.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp038.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.17162709506825813.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp050.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_155_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_158_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_159_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_160_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_61.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.16608165493383653.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_161_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_162_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_164_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.16930709250788165.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_167_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp054.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_0.17152.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/currypurin_173_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_67.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.16852467380720984.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_75.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.16118659547865707.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.1639889734406318.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.16659235221635335.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp062.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp070.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_77.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.15782166199390968.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.15778835194019897.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/currypurin_177_oof.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/currypurin_179_oof.npy',
         #'/content/drive/MyDrive/kaggle_venti/oof/coorabi_exp066_oof.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/rt2kaido_oof_exp75_10folds.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp080.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.15402163834314866.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.14631403817163816.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp081.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_85.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/currypurin_187_oof.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_total_exp75_3.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/currypurin_190_oof.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/currypurin_191_oof.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_88_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cls_cv0.14314169111697705.csv',
         #'/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp085.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_77_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/currypurin_202_oof_wmean.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_93_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/tea_oof_cls_mean_cv0.14481803795874124.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_97.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/coorabi_exp076_oof.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_98_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp090.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_exp75_20folds.npy',
         '/content/drive/MyDrive/kaggle_venti/oof/coorabi_exp078_oof.csv',
         '/content/drive/MyDrive/kaggle_venti/oof/currypurin_209_oof.npy'
        ]



In [36]:
pred_paths = [#'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_cv0.17652843408959074.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_cv0.17823729046894227.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.17542680160619267.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_060_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_062_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_063_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_064_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_065_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.17969709519240776.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/toda_submission_mean_exp070.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_090_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_091_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp022.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp027.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp030.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp031.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp032.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_53_2.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.17454632734884412.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp039.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp040.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp041.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp046.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp047.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp049.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp038.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_53_2.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.17162709506825813.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp050.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_155_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_158_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_159_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_160_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_61.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.16608165493383653.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_161_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_162_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_164_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.16930709250788165.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_167_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp054.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_cv0.17152.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/currypurin_173_test_pred.npy',
         #'/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_67.csv',
         #'/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.16852467380720984.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_75.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.16118659547865707.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.1639889734406318.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.16659235221635335.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp062.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp070.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_77.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.15782166199390968.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.15778835194019897.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/currypurin_177_test_pred.npy',
         '/content/drive/MyDrive/kaggle_venti/sub/currypurin_179_test_pred.npy',
         '/content/drive/MyDrive/kaggle_venti/sub/rt2kaido_submission_exp75_10folds.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp080.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.15402163834314866.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_median_cv0.14631403817163816.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp081.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_85.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/currypurin_187_test_pred.npy',
         '/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp75_3.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/currypurin_190_test_pred.npy',
         '/content/drive/MyDrive/kaggle_venti/sub/currypurin_191_test_pred.npy',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_88_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_cls_median_cv0.14314169111697705.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_77_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/currypurin_202_test_pred_wmean.npy',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_93_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/tea_submission_cls_mean_cv0.14481803795874124.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_97.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/coorabi_exp076_submission_median.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_median_98_10fold.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/yyama_submission_median_exp090.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/rt4kaido_submission_exp75_20folds.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/coorabi_exp078_submission_median.csv',
         '/content/drive/MyDrive/kaggle_venti/sub/currypurin_209_test_pred.npy'
         ]

In [37]:
train = make_oof_features(oof_paths)
test = make_pred_features(pred_paths)

loaded /content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_75.csv
loaded /content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.16118659547865707.csv
loaded /content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.1639889734406318.csv
loaded /content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.16659235221635335.csv
loaded /content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp062.npy
loaded /content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp070.npy
loaded /content/drive/MyDrive/kaggle_venti/oof/rt4kaido_oof_total_77.csv
loaded /content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.15782166199390968.csv
loaded /content/drive/MyDrive/kaggle_venti/oof/tea_oof_cv0.15778835194019897.csv
loaded /content/drive/MyDrive/kaggle_venti/oof/currypurin_177_oof.npy
loaded /content/drive/MyDrive/kaggle_venti/oof/currypurin_179_oof.npy
loaded /content/drive/MyDrive/kaggle_venti/oof/rt2kaido_oof_exp75_10folds.npy
loaded /content/drive/MyDrive/kaggle_venti/oof/yyama_oof_exp080.npy
loaded /content/drive/MyDriv

In [38]:
# save as .npy files
#np.save('/content/drive/MyDrive/kaggle_venti/ens_data/oof_ver1030_3', train)
#np.save('/content/drive/MyDrive/kaggle_venti/ens_data/pred_ver1030_3', test)

In [41]:
oof_col_name = [s.replace('/content/drive/MyDrive/kaggle_venti/oof/', '') for s in oof_paths]
pred_col_name = [s.replace('/content/drive/MyDrive/kaggle_venti/sub/', '') for s in pred_paths]
print(oof_col_name)
print(pred_col_name)

['rt4kaido_oof_total_75.csv', 'tea_oof_cv0.16118659547865707.csv', 'tea_oof_cv0.1639889734406318.csv', 'tea_oof_cv0.16659235221635335.csv', 'yyama_oof_exp062.npy', 'yyama_oof_exp070.npy', 'rt4kaido_oof_total_77.csv', 'tea_oof_cv0.15782166199390968.csv', 'tea_oof_cv0.15778835194019897.csv', 'currypurin_177_oof.npy', 'currypurin_179_oof.npy', 'rt2kaido_oof_exp75_10folds.npy', 'yyama_oof_exp080.npy', 'tea_oof_cv0.15402163834314866.csv', 'tea_oof_cv0.14631403817163816.csv', 'yyama_oof_exp081.npy', 'rt4kaido_oof_total_85.csv', 'currypurin_187_oof.npy', 'yyama_oof_total_exp75_3.csv', 'currypurin_190_oof.npy', 'currypurin_191_oof.npy', 'rt4kaido_oof_total_88_10fold.csv', 'tea_oof_cls_cv0.14314169111697705.csv', 'rt4kaido_oof_total_77_10fold.csv', 'currypurin_202_oof_wmean.npy', 'rt4kaido_oof_total_93_10fold.csv', 'tea_oof_cls_mean_cv0.14481803795874124.csv', 'rt4kaido_oof_total_97.csv', 'coorabi_exp076_oof.csv', 'rt4kaido_oof_total_98_10fold.csv', 'yyama_oof_exp090.npy', 'rt4kaido_oof_exp75_2

In [43]:
pd.DataFrame({'oof':oof_col_name, 'pred':pred_col_name})

,oof,pred
0,rt4kaido_oof_total_75.csv,rt4kaido_submission_median_75.csv
1,tea_oof_cv0.16118659547865707.csv,tea_submission_median_cv0.16118659547865707.csv
2,tea_oof_cv0.1639889734406318.csv,tea_submission_median_cv0.1639889734406318.csv
3,tea_oof_cv0.16659235221635335.csv,tea_submission_median_cv0.16659235221635335.csv
4,yyama_oof_exp062.npy,yyama_submission_median_exp062.csv
5,yyama_oof_exp070.npy,yyama_submission_median_exp070.csv
6,rt4kaido_oof_total_77.csv,rt4kaido_submission_median_77.csv
7,tea_oof_cv0.15782166199390968.csv,tea_submission_median_cv0.15782166199390968.csv
8,tea_oof_cv0.15778835194019897.csv,tea_submission_median_cv0.15778835194019897.csv
9,currypurin_177_oof.npy,currypurin_177_test_pred.npy


In [44]:
tr_df = pd.DataFrame(train)
tr_df.columns = oof_col_name
display(tr_df)
print(tr_df.info())

ts_df = pd.DataFrame(test)
ts_df.columns = pred_col_name
display(ts_df)
print(ts_df.info())

,rt4kaido_oof_total_75.csv,tea_oof_cv0.16118659547865707.csv,tea_oof_cv0.1639889734406318.csv,tea_oof_cv0.16659235221635335.csv,yyama_oof_exp062.npy,yyama_oof_exp070.npy,rt4kaido_oof_total_77.csv,tea_oof_cv0.15782166199390968.csv,tea_oof_cv0.15778835194019897.csv,currypurin_177_oof.npy,currypurin_179_oof.npy,rt2kaido_oof_exp75_10folds.npy,yyama_oof_exp080.npy,tea_oof_cv0.15402163834314866.csv,tea_oof_cv0.14631403817163816.csv,yyama_oof_exp081.npy,rt4kaido_oof_total_85.csv,currypurin_187_oof.npy,yyama_oof_total_exp75_3.csv,currypurin_190_oof.npy,currypurin_191_oof.npy,rt4kaido_oof_total_88_10fold.csv,tea_oof_cls_cv0.14314169111697705.csv,rt4kaido_oof_total_77_10fold.csv,currypurin_202_oof_wmean.npy,rt4kaido_oof_total_93_10fold.csv,tea_oof_cls_mean_cv0.14481803795874124.csv,rt4kaido_oof_total_97.csv,coorabi_exp076_oof.csv,rt4kaido_oof_total_98_10fold.csv,yyama_oof_exp090.npy,rt4kaido_oof_exp75_20folds.npy,coorabi_exp078_oof.csv,currypurin_209_oof.npy
0,6.094115,5.788125,5.769589,5.660350,5.793085,5.843223,6.156596,5.791456,5.767539,5.783708,6.735515,6.287325,5.776286,5.808908,5.798063,5.868944,5.784706,5.903346,5.895987,5.868830,5.828472,5.798295,5.783465,6.013763,6.251595,5.788308,5.823515,6.477747,5.843684,7.629176,5.816144,6.452471,5.834950,6.206788
1,5.988505,5.747135,5.817975,5.725365,5.832949,5.873457,6.049686,5.939394,5.933484,5.767036,5.966020,5.989612,5.852838,5.905727,5.848358,5.877565,5.817481,5.912168,6.107488,5.866003,5.880560,5.852804,5.869486,6.103593,5.920428,5.954981,5.898210,6.534627,5.854125,6.086765,5.794350,5.884715,5.841093,5.789935
2,8.814715,7.770787,7.828234,7.785930,7.865380,7.892688,8.371438,7.811497,7.767039,7.797904,8.667629,8.725241,7.893561,7.914346,7.790454,7.873539,7.859506,8.184838,8.506552,8.075563,8.157262,7.833846,7.865620,8.852818,8.440607,8.644610,7.930587,8.846078,7.914377,8.957220,7.781632,8.618551,7.889086,8.631878
3,12.582986,11.665780,11.786056,11.642105,11.782902,11.861191,12.139318,11.833604,11.905822,11.810205,12.377403,12.752718,11.892482,11.722112,11.766545,11.914234,11.914021,12.292066,12.863997,12.167562,12.129442,11.784959,11.779996,12.312957,12.190743,12.742400,12.042971,12.801433,11.954165,12.609935,11.814193,12.376060,11.697797,12.434288
4,13.164696,12.434586,12.307502,12.144012,12.286124,12.331974,12.431278,12.376520,12.410877,12.356912,12.624374,12.974612,12.380159,12.275325,12.218771,12.443951,12.533509,12.780018,13.025760,12.675638,12.576923,12.328327,12.318006,12.656638,12.622059,13.086274,12.474959,13.259793,12.393296,12.761421,12.382209,12.714886,12.329643,12.796815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035995,8.846284,17.643142,8.269836,28.954405,3.931680,3.878536,6.428029,38.118529,30.006470,35.273590,29.113579,10.053699,3.937903,34.565862,27.399870,3.918233,10.789883,42.256641,22.079739,37.528717,33.113632,9.102180,23.812657,9.911885,9.750838,7.588490,50.085614,7.869389,3.867466,14.699379,3.898078,8.355096,3.899063,9.972795
6035996,10.022511,18.046538,9.959403,29.599184,3.926268,3.888600,7.051559,38.106152,29.859782,26.280306,18.889898,10.087551,3.938724,34.934957,29.374889,3.927953,11.333364,42.253529,19.342186,38.332512,33.605228,9.244560,23.587521,9.934764,9.728185,7.848327,49.767257,11.205536,3.872056,14.747525,3.905671,8.279023,3.901155,9.939343
6035997,7.912016,17.785429,8.537014,30.917606,3.865344,3.815409,8.029111,37.784642,29.695158,9.626728,35.009052,9.693629,3.865407,33.771788,27.324729,3.841502,9.917234,43.633011,18.306789,39.062355,33.979126,8.391934,22.652757,9.812245,9.473552,7.809254,32.764555,11.790893,3.791186,17.724089,3.847541,8.294396,3.842515,9.892908
6035998,10.080069,20.250578,21.169361,32.344128,4.197916,4.180053,9.558969,41.512899,30.611357,7.703361,9.251776,10.223997,4.174096,35.928327,31.433994,4.194509,10.822178,42.616627,10.601183,36.353985,31.554214,9.415244,26.580203,8.975171,10.027510,9.733979,27.734379,10.896358,4.091510,7.662900,

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6036000 entries, 0 to 6035999
Data columns (total 34 columns):
 #   Column                                      Dtype  
---  ------                                      -----  
 0   rt4kaido_oof_total_75.csv                   float64
 1   tea_oof_cv0.16118659547865707.csv           float64
 2   tea_oof_cv0.1639889734406318.csv            float64
 3   tea_oof_cv0.16659235221635335.csv           float64
 4   yyama_oof_exp062.npy                        float64
 5   yyama_oof_exp070.npy                        float64
 6   rt4kaido_oof_total_77.csv                   float64
 7   tea_oof_cv0.15782166199390968.csv           float64
 8   tea_oof_cv0.15778835194019897.csv           float64
 9   currypurin_177_oof.npy                      float64
 10  currypurin_179_oof.npy                      float64
 11  rt2kaido_oof_exp75_10folds.npy              float64
 12  yyama_oof_exp080.npy                        float64
 13  tea_oof_cv0.154021638343148

,rt4kaido_submission_median_75.csv,tea_submission_median_cv0.16118659547865707.csv,tea_submission_median_cv0.1639889734406318.csv,tea_submission_median_cv0.16659235221635335.csv,yyama_submission_median_exp062.csv,yyama_submission_median_exp070.csv,rt4kaido_submission_median_77.csv,tea_submission_median_cv0.15782166199390968.csv,tea_submission_median_cv0.15778835194019897.csv,currypurin_177_test_pred.npy,currypurin_179_test_pred.npy,rt2kaido_submission_exp75_10folds.csv,yyama_submission_median_exp080.csv,tea_submission_median_cv0.15402163834314866.csv,tea_submission_median_cv0.14631403817163816.csv,yyama_submission_median_exp081.csv,rt4kaido_submission_median_85.csv,currypurin_187_test_pred.npy,yyama_submission_median_exp75_3.csv,currypurin_190_test_pred.npy,currypurin_191_test_pred.npy,rt4kaido_submission_median_88_10fold.csv,tea_submission_cls_median_cv0.14314169111697705.csv,rt4kaido_submission_median_77_10fold.csv,currypurin_202_test_pred_wmean.npy,rt4kaido_submission_median_93_10fold.csv,tea_submission_cls_mean_cv0.14481803795874124.csv,rt4kaido_submission_median_97.csv,coorabi_exp076_submission_median.csv,rt4kaido_submission_median_98_10fold.csv,yyama_submission_median_exp090.csv,rt4kaido_submission_exp75_20folds.csv,coorabi_exp078_submission_median.csv,currypurin_209_test_pred.npy
0,6.206704,6.300555,6.269852,6.299540,6.246757,6.237001,4.641518,6.297037,6.282405,5.904403,6.168560,6.190098,6.230516,6.327006,6.286294,6.230786,6.273802,6.245049,6.194475,6.226122,6.258121,6.274621,6.315998,6.157633,6.153747,6.123875,6.301814,6.130114,6.295876,6.113052,6.245053,6.208011,6.291997,6.191348
1,5.991534,5.958212,5.932146,5.982564,5.911918,5.938651,4.805767,5.941913,5.957493,5.620617,6.049200,6.075787,5.919991,5.938365,5.926586,5.932162,5.938371,5.945202,6.016113,5.946875,5.953184,5.936455,5.938564,6.029568,6.061356,6.048179,5.943007,6.060609,5.965106,5.969356,5.935810,6.033892,5.990556,6.055338
2,7.201249,7.091644,7.123506,7.189820,7.172530,7.218726,6.455574,7.105504,7.121612,6.863608,7.145738,7.236039,7.180171,7.111823,7.109683,7.139564,7.121479,7.177931,7.118916,7.179124,7.180656,7.120364,7.115508,7.136393,7.312749,7.161344,7.136607,7.189013,7.117581,7.113848,7.179698,7.160792,7.142857,7.202180
3,7.794795,7.598707,7.609760,7.624099,7.763091,7.752637,7.217239,7.629017,7.620585,7.506460,7.714159,7.880373,7.763195,7.615339,7.632365,7.725376,7.587220,7.810214,7.738754,7.781487,7.789594,7.610859,7.607679,7.800501,7.841540,7.809066,7.651355,7.909873,7.631863,7.706476,7.734634,7.800667,7.654799,7.776278
4,9.245929,9.106483,9.130349,9.185045,9.325331,9.262370,8.986800,9.095490,9.137233,9.083950,9.273138,9.373093,9.265894,9.106141,9.117007,9.212476,9.093376,9.301676,9.282778,9.286556,9.275614,9.124668,9.145709,9.164462,9.336922,9.351730,9.160076,9.389410,9.132385,9.256947,9.260639,9.270784,9.161454,9.304310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023995,16.857979,20.561100,18.898885,33.602013,6.197637,6.187890,17.061760,14.022980,13.909360,40.755890,16.502970,14.774168,6.240428,15.842274,13.814573,6.178785,14.394050,38.859104,14.954706,36.886230,35.090607,14.169148,15.863170,18.102705,16.602873,14.478188,18.055397,11.708288,6.150437,12.287502,6.208994,14.476089,6.163520,16.766878
4023996,17.300259,20.998857,19.529404,32.883360,6.259319,6.290313,16.432041,14.207393,14.335769,37.674564,17.265522,14.924815,6.230602,16.356199,14.047708,6.274732,14.660531,40.331650,15.136181,37.757469,35.653534,14.360403,16.148630,18.367050,17.279026,14.724933,20.852467,11.723017,6.322860,12.271587,6.269710,14.706350,6.320943,16.987122
4023997,17.354898,21.251579,20.276822,31.600544,6.401722,6.397727,15.802923,14.365244,14.447198,39.071018,15.154324,14.827735,6.464466,15.944096,13.941481,6.414190,14.871182,41.759369,15.379525,38.659023,36.066963,14.503628,16.197741,17.861053,17.451265,14.820187,22.164679,12.104719,6.358964,12.555499,6.435824,14.665152,6.379314

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024000 entries, 0 to 4023999
Data columns (total 34 columns):
 #   Column                                               Dtype  
---  ------                                               -----  
 0   rt4kaido_submission_median_75.csv                    float64
 1   tea_submission_median_cv0.16118659547865707.csv      float64
 2   tea_submission_median_cv0.1639889734406318.csv       float64
 3   tea_submission_median_cv0.16659235221635335.csv      float64
 4   yyama_submission_median_exp062.csv                   float64
 5   yyama_submission_median_exp070.csv                   float64
 6   rt4kaido_submission_median_77.csv                    float64
 7   tea_submission_median_cv0.15782166199390968.csv      float64
 8   tea_submission_median_cv0.15778835194019897.csv      float64
 9   currypurin_177_test_pred.npy                         float64
 10  currypurin_179_test_pred.npy                         float64
 11  rt2kaido_submission_exp7

In [ ]:
tr_df.to_csv('/content/drive/MyDrive/kaggle_venti/ens_data/oof_ver1031.csv', index=False)
ts_df.to_csv('/content/drive/MyDrive/kaggle_venti/ens_data/pred_ver1031.csv', index=False)